# Appendices

In [1]:
import numpy as np
import statsmodels.api as sm

In [2]:
from joblib import Parallel, delayed

## A1: bootstrap

In [12]:
n, MC, nboot = 500, 1000, 500


def onerep():
    Y = np.random.normal(1, 1, n)
    oneboot = lambda: np.mean(Y[np.random.randint(0, n, n)])
    boot_mu_hat = np.array([oneboot() for _ in range(nboot)])
    return np.mean(Y), np.var(Y, ddof=1) / n, np.var(boot_mu_hat, ddof=1)


onerep()

(0.9889370765217561, 0.002111898300454035, 0.0021215077032831677)

In [15]:
res = Parallel(n_jobs=-1)(delayed(onerep)() for _ in range(MC))
res = np.array(res)
np.round(res.mean(axis=0), 3)

array([1.002, 0.002, 0.002])

## A2: Robust SEs

In [21]:
import statsmodels.formula.api as smf

In [52]:
boston = pd.read_csv(
    "https://raw.githubusercontent.com/selva86/datasets/master/BostonHousing.csv"
)
boston.describe()

,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,b,lstat,medv
count,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000
mean,3.613524,11.363636,11.136779,0.069170,0.554695,6.284634,68.574901,3.795043,9.549407,408.237154,18.455534,356.674032,12.653063,22.532806
std,8.601545,23.322453,6.860353,0.253994,0.115878,0.702617,28.148861,2.105710,8.707259,168.537116,2.164946,91.294864,7.141062,9.197104
min,0.006320,0.000000,0.460000,0.000000,0.385000,3.561000,2.900000,1.129600,1.000000,187.000000,12.600000,0.320000,1.730000,5.000000
25%,0.082045,0.000000,5.190000,0.000000,0.449000,5.885500,45.025000,2.100175,4.000000,279.000000,17.400000,375.377500,6.950000,17.025000
50%,0.256510,0.000000,9.690000,0.000000,0.538000,6.208500,77.500000,3.207450,5.000000,330.000000,19.050000,391.440000,11.360000,21.200000
75%,3.677083,12.500000,18.100000,0.000000,0.624000,6.623500,94.075000,5.188425,24.000000,666.000000,20.200000,396.225000,16.955000,25.000000
max,88.976200,100.000000,27.740000,1.000000,0.871000,8.780000,100.000000,12.126500,24.000000,711.000000,22.000000,396.900000,37.970000,50.000000


In [29]:
(f := f"medv ~ {'+'.join(boston.columns[:-1])}")

'medv ~ crim+zn+indus+chas+nox+rm+age+dis+rad+tax+ptratio+b+lstat'

In [49]:
m = smf.ols(f, data=boston)
coef = m.fit().params
se_types = ["HC0", "HC1", "HC2", "HC3"]
SEs = [m.fit(cov_type=x).bse for x in se_types]

pd.DataFrame(np.c_[coef, *SEs], columns=["coef", *se_types], index=m.exog_names).round(
    3
)

,coef,HC0,HC1,HC2,HC3
Intercept,36.459,7.890,8.001,8.145,8.412
crim,-0.108,0.029,0.029,0.031,0.034
zn,0.046,0.014,0.014,0.014,0.014
indus,0.021,0.050,0.050,0.051,0.052
chas,2.687,1.276,1.294,1.310,1.345
nox,-17.767,3.733,3.786,3.827,3.924
rm,3.810,0.833,0.845,0.861,0.890
age,0.001,0.016,0.016,0.017,0.017
dis,-1.476,0.212,0.215,0.217,0.222
rad,0.306,0.061,0.061,0.062,0.064
